In the "StationAnalysis.ipynb," I calculated the N of rides > 1 hour for member and casual users. But it isn't clear what percent of ALL rides these numbers represent. So this notebook will figure that information out. The goal is to figure out when we've acccount for 95% (or 99%) of all rides and can justify ignoring rides longer than X hours.